In [62]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from random import randint
%matplotlib inline 

from sklearn.model_selection import train_test_split
def split(df,label):
    X_tr, X_te, Y_tr, Y_te = train_test_split(df, label, test_size=0.25, random_state=42)
    return X_tr, X_te, Y_tr, Y_te

from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold, cross_val_score
from sklearn.naive_bayes import GaussianNB

classifiers = ['LinearSVM', 'RadialSVM', 
               'Logistic',  'RandomForest', 
               'AdaBoost',  'DecisionTree', 
               'KNeighbors','GradientBoosting', 'NaiveBayes']

models = [svm.SVC(kernel='linear'),
          svm.SVC(kernel='rbf'),
          LogisticRegression(max_iter = 1000),
          RandomForestClassifier(n_estimators=200, random_state=0),
          AdaBoostClassifier(random_state = 0),
          DecisionTreeClassifier(random_state=0),
          KNeighborsClassifier(),
          GradientBoostingClassifier(random_state=0),
          GaussianNB()]


def acc_score(df,label):
    Score = pd.DataFrame({"Classifier":classifiers})
    j = 0
    acc = []
    X_train,X_test,Y_train,Y_test = split(df,label)
    for i in models:
        model = i
        model.fit(X_train,Y_train)
        predictions = model.predict(X_test)
        acc.append(accuracy_score(Y_test,predictions))
        j = j+1     
    Score["Accuracy"] = acc
    Score.sort_values(by="Accuracy", ascending=False,inplace = True)
    Score.reset_index(drop=True, inplace=True)
    return Score

def plot(score,x,y,c = "b"):
    gen = [1,2,3,4,5]
    plt.figure(figsize=(6,4))
    ax = sns.pointplot(x=gen, y=score,color = c )
    ax.set(xlabel="Generation", ylabel="Accuracy")
    ax.set(ylim=(x,y))

In [63]:
def initilization_of_population(size,n_feat):
    population = []
    for i in range(size):
        chromosome = np.ones(n_feat,dtype=np.bool)     
        chromosome[:int(0.3*n_feat)]=False             
        np.random.shuffle(chromosome)
        population.append(chromosome)
    return population


def fitness_score(population):
    scores = []
    for chromosome in population:
        logmodel.fit(X_train.iloc[:,chromosome],Y_train)         
        predictions = logmodel.predict(X_test.iloc[:,chromosome])
        scores.append(accuracy_score(Y_test,predictions))
    scores, population = np.array(scores), np.array(population) 
    inds = np.argsort(scores)                                    
    return list(scores[inds][::-1]), list(population[inds,:][::-1]) 


def selection(pop_after_fit,n_parents):
    population_nextgen = []
    for i in range(n_parents):
        population_nextgen.append(pop_after_fit[i])
    return population_nextgen


def crossover(pop_after_sel):
    pop_nextgen = pop_after_sel
    for i in range(0,len(pop_after_sel),2):
        new_par = []
        child_1 , child_2 = pop_nextgen[i] , pop_nextgen[i+1]
        new_par = np.concatenate((child_1[:len(child_1)//2],child_2[len(child_1)//2:]))
        pop_nextgen.append(new_par)
    return pop_nextgen


def mutation(pop_after_cross,mutation_rate,n_feat):   
    mutation_range = int(mutation_rate*n_feat)
    pop_next_gen = []
    for n in range(0,len(pop_after_cross)):
        chromo = pop_after_cross[n]
        rand_posi = [] 
        for i in range(0,mutation_range):
            pos = randint(0,n_feat-1)
            rand_posi.append(pos)
        for j in rand_posi:
            chromo[j] = not chromo[j]  
        pop_next_gen.append(chromo)
    return pop_next_gen

def generations(df,label,size,n_feat,n_parents,mutation_rate,n_gen,X_train,
                                   X_test, Y_train, Y_test):
    best_chromo= []
    best_score= []
    population_nextgen=initilization_of_population(size,n_feat)
    for i in range(n_gen):
        scores, pop_after_fit = fitness_score(population_nextgen)
        print('Best score in generation',i+1,':',scores[:1])  #2
        pop_after_sel = selection(pop_after_fit,n_parents)
        pop_after_cross = crossover(pop_after_sel)
        population_nextgen = mutation(pop_after_cross,mutation_rate,n_feat)
        best_chromo.append(pop_after_fit[0])
        best_score.append(scores[0])
    return best_chromo,best_score

In [64]:
data_bc = pd.read_csv("diabetes_data_upload.csv")
label_bc = data_bc["class"]
print(data_bc.describe())
#label_bc = np.where(label_bc == 'M',1,0)
data_bc.drop(["class"],axis = 1,inplace = True)

print("Diabetes dataset:\n",data_bc.shape[0],"Records\n",data_bc.shape[1],"Features")
data_bc.describe()

              Age       class
count  520.000000  520.000000
mean    48.028846    0.615385
std     12.151466    0.486973
min     16.000000    0.000000
25%     39.000000    0.000000
50%     47.500000    1.000000
75%     57.000000    1.000000
max     90.000000    1.000000
Diabetes dataset:
 520 Records
 16 Features


,Age
count,520.000000
mean,48.028846
std,12.151466
min,16.000000
25%,39.000000
50%,47.500000
75%,57.000000
max,90.000000


In [65]:
data_bc['Gender'] = data_bc['Gender'].map({'Male':1,'Female':0})
data_bc['Polyuria'] = data_bc['Polyuria'].map({'Yes':1,'No':0})
data_bc['Polydipsia'] = data_bc['Polydipsia'].map({'Yes':1,'No':0})
data_bc['sudden weight loss'] = data_bc['sudden weight loss'].map({'Yes':1,'No':0})
data_bc['weakness'] = data_bc['weakness'].map({'Yes':1,'No':0})
data_bc['Polyphagia'] = data_bc['Polyphagia'].map({'Yes':1,'No':0})
data_bc['Genital thrush'] = data_bc['Genital thrush'].map({'Yes':1,'No':0})
data_bc['visual blurring'] = data_bc['visual blurring'].map({'Yes':1,'No':0})
data_bc['Itching'] = data_bc['Itching'].map({'Yes':1,'No':0})
data_bc['Irritability'] = data_bc['Irritability'].map({'Yes':1,'No':0})
data_bc['delayed healing'] = data_bc['delayed healing'].map({'Yes':1,'No':0})
data_bc['partial paresis'] = data_bc['partial paresis'].map({'Yes':1,'No':0})
data_bc['muscle stiffness'] = data_bc['muscle stiffness'].map({'Yes':1,'No':0})
data_bc['Alopecia'] = data_bc['Alopecia'].map({'Yes':1,'No':0})
data_bc['Obesity'] = data_bc['Obesity'].map({'Yes':1,'No':0})

data_bc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 520 entries, 0 to 519
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   Age                 520 non-null    int64
 1   Gender              520 non-null    int64
 2   Polyuria            520 non-null    int64
 3   Polydipsia          520 non-null    int64
 4   sudden weight loss  520 non-null    int64
 5   weakness            520 non-null    int64
 6   Polyphagia          520 non-null    int64
 7   Genital thrush      520 non-null    int64
 8   visual blurring     520 non-null    int64
 9   Itching             520 non-null    int64
 10  Irritability        520 non-null    int64
 11  delayed healing     520 non-null    int64
 12  partial paresis     520 non-null    int64
 13  muscle stiffness    520 non-null    int64
 14  Alopecia            520 non-null    int64
 15  Obesity             520 non-null    int64
dtypes: int64(16)
memory usage: 65.1 KB


In [66]:
# label_bc['class'] = label_bc['class'].map({'Positive':1,'Negative':0})
label_bc

0      1
1      1
2      1
3      1
4      1
      ..
515    1
516    1
517    1
518    0
519    0
Name: class, Length: 520, dtype: int64

In [67]:
display(data_bc.head())

,Age,Gender,Polyuria,Polydipsia,sudden weight loss,weakness,Polyphagia,Genital thrush,visual blurring,Itching,Irritability,delayed healing,partial paresis,muscle stiffness,Alopecia,Obesity
0,40,1,0,1,0,1,0,0,0,1,0,1,0,1,1,1
1,58,1,0,0,0,1,0,0,1,0,0,0,1,0,1,0
2,41,1,1,0,0,1,1,0,0,1,0,1,0,1,1,0
3,45,1,0,0,1,1,1,1,0,1,0,1,0,0,0,0
4,60,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1


In [68]:
score1 = acc_score(data_bc,label_bc)
score1

,Classifier,Accuracy
0,RandomForest,0.992308
1,GradientBoosting,0.976923
2,DecisionTree,0.969231
3,AdaBoost,0.946154
4,Logistic,0.930769
5,LinearSVM,0.923077
6,NaiveBayes,0.915385
7,KNeighbors,0.846154
8,RadialSVM,0.646154


In [69]:
# logmodel = RandomForestClassifier(n_estimators=200, random_state=0)
# X_train,X_test, Y_train, Y_test = split(data_bc,label_bc)
# chromo_df_bc,score_bc=generations(data_bc,label_bc,size=80,n_feat=data_bc.shape[1],n_parents=64,mutation_rate=0.20,n_gen=500,
#                          X_train = X_train,X_test = X_test,Y_train = Y_train,Y_test = Y_test)

# logmodel = AdaBoostClassifier(random_state = 0)
# X_train,X_test, Y_train, Y_test = split(data_bc,label_bc)
# chromo_df_bc,score_bc=generations(data_bc,label_bc,size=80,n_feat=data_bc.shape[1],n_parents=64,mutation_rate=0.20,n_gen=500,
#                          X_train = X_train,X_test = X_test,Y_train = Y_train,Y_test = Y_test)

# logmodel = GaussianNB()
# X_train,X_test, Y_train, Y_test = split(data_bc,label_bc)
# chromo_df_bc,score_bc=generations(data_bc,label_bc,size=80,n_feat=data_bc.shape[1],n_parents=64,mutation_rate=0.20,n_gen=500,
#                          X_train = X_train,X_test = X_test,Y_train = Y_train,Y_test = Y_test)

# logmodel = LogisticRegression(max_iter = 1000)
# X_train,X_test, Y_train, Y_test = split(data_bc,label_bc)
# chromo_df_bc,score_bc=generations(data_bc,label_bc,size=80,n_feat=data_bc.shape[1],n_parents=64,mutation_rate=0.20,n_gen=500,
#                          X_train = X_train,X_test = X_test,Y_train = Y_train,Y_test = Y_test)

logmodel = DecisionTreeClassifier(random_state=0)
X_train,X_test, Y_train, Y_test = split(data_bc,label_bc)
chromo_df_bc,score_bc=generations(data_bc,label_bc,size=80,n_feat=data_bc.shape[1],n_parents=64,mutation_rate=0.20,n_gen=500,
                         X_train = X_train,X_test = X_test,Y_train = Y_train,Y_test = Y_test)

# logmodel = KNeighborsClassifier()
# X_train,X_test, Y_train, Y_test = split(data_bc,label_bc)
# chromo_df_bc,score_bc=generations(data_bc,label_bc,size=80,n_feat=data_bc.shape[1],n_parents=64,mutation_rate=0.20,n_gen=500,
#                          X_train = X_train,X_test = X_test,Y_train = Y_train,Y_test = Y_test)

# logmodel = GradientBoostingClassifier(random_state=0)
# X_train,X_test, Y_train, Y_test = split(data_bc,label_bc)
# chromo_df_bc,score_bc=generations(data_bc,label_bc,size=80,n_feat=data_bc.shape[1],n_parents=64,mutation_rate=0.20,n_gen=500,
#                          X_train = X_train,X_test = X_test,Y_train = Y_train,Y_test = Y_test)

<ipython-input-63-0f6289eb5f5e>:4: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  chromosome = np.ones(n_feat,dtype=np.bool)


Best score in generation 1 : [0.9230769230769231]
Best score in generation 2 : [0.9230769230769231]
Best score in generation 3 : [0.9230769230769231]
Best score in generation 4 : [0.9307692307692308]
Best score in generation 5 : [0.9230769230769231]
Best score in generation 6 : [0.9307692307692308]
Best score in generation 7 : [0.9230769230769231]
Best score in generation 8 : [0.9230769230769231]
Best score in generation 9 : [0.9307692307692308]
Best score in generation 10 : [0.9307692307692308]
Best score in generation 11 : [0.9307692307692308]
Best score in generation 12 : [0.9230769230769231]
Best score in generation 13 : [0.9230769230769231]
Best score in generation 14 : [0.9230769230769231]
Best score in generation 15 : [0.9230769230769231]
Best score in generation 16 : [0.9307692307692308]
Best score in generation 17 : [0.9307692307692308]
Best score in generation 18 : [0.9230769230769231]
Best score in generation 19 : [0.9230769230769231]
Best score in generation 20 : [0.9307692

Best score in generation 161 : [0.9230769230769231]
Best score in generation 162 : [0.9307692307692308]
Best score in generation 163 : [0.9307692307692308]
Best score in generation 164 : [0.9384615384615385]
Best score in generation 165 : [0.9307692307692308]
Best score in generation 166 : [0.9230769230769231]
Best score in generation 167 : [0.9307692307692308]
Best score in generation 168 : [0.9307692307692308]
Best score in generation 169 : [0.9307692307692308]
Best score in generation 170 : [0.9230769230769231]
Best score in generation 171 : [0.9230769230769231]
Best score in generation 172 : [0.9230769230769231]
Best score in generation 173 : [0.9230769230769231]
Best score in generation 174 : [0.9230769230769231]
Best score in generation 175 : [0.9230769230769231]
Best score in generation 176 : [0.9307692307692308]
Best score in generation 177 : [0.9307692307692308]
Best score in generation 178 : [0.9307692307692308]
Best score in generation 179 : [0.9230769230769231]
Best score i

Best score in generation 319 : [0.9230769230769231]
Best score in generation 320 : [0.9230769230769231]
Best score in generation 321 : [0.9307692307692308]
Best score in generation 322 : [0.9230769230769231]
Best score in generation 323 : [0.9307692307692308]
Best score in generation 324 : [0.9230769230769231]
Best score in generation 325 : [0.9384615384615385]
Best score in generation 326 : [0.9230769230769231]
Best score in generation 327 : [0.9307692307692308]
Best score in generation 328 : [0.9230769230769231]
Best score in generation 329 : [0.9230769230769231]
Best score in generation 330 : [0.9307692307692308]
Best score in generation 331 : [0.9230769230769231]
Best score in generation 332 : [0.9307692307692308]
Best score in generation 333 : [0.9307692307692308]
Best score in generation 334 : [0.9230769230769231]
Best score in generation 335 : [0.9307692307692308]
Best score in generation 336 : [0.9230769230769231]
Best score in generation 337 : [0.9230769230769231]
Best score i

Best score in generation 477 : [0.9230769230769231]
Best score in generation 478 : [0.9230769230769231]
Best score in generation 479 : [0.9307692307692308]
Best score in generation 480 : [0.9307692307692308]
Best score in generation 481 : [0.9384615384615385]
Best score in generation 482 : [0.9230769230769231]
Best score in generation 483 : [0.9230769230769231]
Best score in generation 484 : [0.9307692307692308]
Best score in generation 485 : [0.9307692307692308]
Best score in generation 486 : [0.9230769230769231]
Best score in generation 487 : [0.9307692307692308]
Best score in generation 488 : [0.9230769230769231]
Best score in generation 489 : [0.9307692307692308]
Best score in generation 490 : [0.9307692307692308]
Best score in generation 491 : [0.9307692307692308]
Best score in generation 492 : [0.9384615384615385]
Best score in generation 493 : [0.9307692307692308]
Best score in generation 494 : [0.9230769230769231]
Best score in generation 495 : [0.9230769230769231]
Best score i

In [70]:
chromo_df_bc

[array([False,  True,  True,  True,  True,  True,  True,  True,  True,
        False,  True, False,  True, False,  True, False]),
 array([False,  True,  True, False,  True,  True,  True,  True, False,
        False,  True, False,  True,  True,  True, False]),
 array([ True,  True,  True,  True, False,  True,  True, False,  True,
        False,  True,  True,  True,  True, False, False]),
 array([ True, False, False,  True, False, False, False, False, False,
        False, False, False, False, False,  True, False]),
 array([ True,  True,  True,  True,  True, False,  True,  True,  True,
         True, False,  True, False, False, False,  True]),
 array([ True,  True, False,  True,  True,  True,  True,  True,  True,
        False, False, False,  True, False, False, False]),
 array([ True,  True,  True,  True, False, False, False,  True, False,
         True, False, False,  True, False,  True, False]),
 array([False,  True,  True,  True,  True,  True, False, False, False,
        False, Fals

In [71]:
dataset = pd.read_csv('diabetes_data_upload.csv')

dataset['Gender'] = dataset['Gender'].map({'Male':1,'Female':0})
# dataset['class'] = dataset['class'].map({'Positive':1,'Negative':0})
dataset['Polyuria'] = dataset['Polyuria'].map({'Yes':1,'No':0})
dataset['Polydipsia'] = dataset['Polydipsia'].map({'Yes':1,'No':0})
dataset['sudden weight loss'] = dataset['sudden weight loss'].map({'Yes':1,'No':0})
dataset['weakness'] = dataset['weakness'].map({'Yes':1,'No':0})
dataset['Polyphagia'] = dataset['Polyphagia'].map({'Yes':1,'No':0})
dataset['Genital thrush'] = dataset['Genital thrush'].map({'Yes':1,'No':0})
dataset['visual blurring'] = dataset['visual blurring'].map({'Yes':1,'No':0})
dataset['Itching'] = dataset['Itching'].map({'Yes':1,'No':0})
dataset['Irritability'] = dataset['Irritability'].map({'Yes':1,'No':0})
dataset['delayed healing'] = dataset['delayed healing'].map({'Yes':1,'No':0})
dataset['partial paresis'] = dataset['partial paresis'].map({'Yes':1,'No':0})
dataset['muscle stiffness'] = dataset['muscle stiffness'].map({'Yes':1,'No':0})
dataset['Alopecia'] = dataset['Alopecia'].map({'Yes':1,'No':0})
dataset['Obesity'] = dataset['Obesity'].map({'Yes':1,'No':0})

dataset

,Age,Gender,Polyuria,Polydipsia,sudden weight loss,weakness,Polyphagia,Genital thrush,visual blurring,Itching,Irritability,delayed healing,partial paresis,muscle stiffness,Alopecia,Obesity,class
0,40,1,0,1,0,1,0,0,0,1,0,1,0,1,1,1,1
1,58,1,0,0,0,1,0,0,1,0,0,0,1,0,1,0,1
2,41,1,1,0,0,1,1,0,0,1,0,1,0,1,1,0,1
3,45,1,0,0,1,1,1,1,0,1,0,1,0,0,0,0,1
4,60,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515,39,0,1,1,1,0,1,0,0,1,0,1,1,0,0,0,1
516,48,0,1,1,1,1,1,0,0,1,1,1,1,0,0,0,1
517,58,0,1,1,1,1,1,0,1,0,0,0,1,1,0,1,1
518,32,0,0,0,0,1,0,0,1,1,0,1,0,0,1,0,0


In [72]:
# X = pd.DataFrame(dataset.iloc[:,[0,1,3,4,6,9,10,11,15]].values)
# y = pd.DataFrame(dataset.iloc[:, -1].values)

# X

In [73]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [74]:
# from sklearn.preprocessing import StandardScaler
# sc = StandardScaler()
# X_train = sc.fit_transform(X_train)
# X_test = sc.transform(X_test)

In [75]:
# X = pd.DataFrame(dataset.iloc[:,[0,1,3,4,6,9,10,11,15]].values)
# y = pd.DataFrame(dataset.iloc[:, -1].values)

In [76]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [77]:
# from sklearn.linear_model import LogisticRegression
# classifier = LogisticRegression(random_state = 0)
# classifier.fit(X_train, y_train)

In [78]:
# y_pred = classifier.predict(X_test)
# y_pred = np.array(y_pred)
# y_test = np.array(y_test)
# print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

In [79]:
# from sklearn.metrics import confusion_matrix, accuracy_score
# cm = confusion_matrix(y_test, y_pred)
# print(cm)
# accuracy_score(y_test, y_pred)

In [80]:
# classifier.predict([[53,1,0,0,1,1,0,1,1]])

In [81]:
X = pd.DataFrame(dataset.iloc[:,[1,2,3,4,8,12,13,14,15]].values)
y = pd.DataFrame(dataset.iloc[:, -1].values)

In [82]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [83]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [84]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)

C:\Users\91779\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


GaussianNB()

In [85]:
y_pred = classifier.predict(X_test)
y_pred = np.array(y_pred)
y_test = np.array(y_test)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[1 1]
 [1 1]
 [1 1]
 [0 0]
 [0 0]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [0 0]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [0 0]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [0 0]
 [0 0]
 [0 0]
 [1 1]
 [1 1]
 [1 0]
 [1 1]
 [1 1]
 [0 0]
 [0 0]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [0 0]
 [1 1]
 [1 1]
 [1 1]
 [0 0]
 [1 1]
 [1 0]
 [0 0]
 [1 1]
 [1 1]
 [0 0]
 [1 1]
 [0 0]
 [1 1]
 [0 0]
 [1 1]
 [0 0]
 [1 1]
 [0 1]
 [1 1]
 [0 1]
 [1 1]
 [1 1]
 [0 0]
 [0 0]
 [1 1]
 [0 0]
 [1 1]
 [1 0]
 [1 1]
 [1 0]
 [1 1]
 [1 1]
 [0 0]
 [0 0]
 [1 1]
 [1 1]
 [0 0]
 [1 1]
 [0 0]
 [1 1]
 [1 1]
 [0 0]
 [1 1]
 [1 1]
 [0 0]
 [0 0]
 [1 1]
 [1 0]
 [1 1]
 [1 1]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [0 0]
 [0 0]
 [1 1]
 [1 1]
 [1 1]
 [0 0]
 [0 0]
 [1 1]
 [0 0]
 [1 1]
 [0 0]
 [1 0]
 [0 0]
 [1 1]
 [0 0]
 [1 1]
 [0 1]
 [0 0]
 [1 1]
 [0 0]
 [0 0]
 [0 0]
 [0 1]
 [1 1]
 [0 0]
 [1 1]
 [1 1]
 [0 0]
 [0 0]
 [0 0]
 [0 0]]


In [86]:
classifier.predict([[0,1,0,1,0,0,1,0,1]])

array([1], dtype=int64)

In [87]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[50  6]
 [ 4 70]]


0.9230769230769231